In [1]:
import h2o
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### Overview
[Reference](https://github.com/NhanDoV/Lectures_notes-teaching-in-VN-/blob/master/h2o_AutoML/classification/clf-tradml-vs-automl.ipynb)

In [2]:
import warnings
warnings.filterwarnings("ignore")
df = sns.load_dataset("titanic")

df.fillna(value={"age": int(df["age"].median())}, inplace=True)
df['survived'] = df['survived'].replace({0: 'dead', 1:'alived'})
df = df.dropna(axis=0, subset='embark_town')
df['alone'] = df['alone'].astype(str)
df['age'] = df['age'].astype(int)

for col in ['fare', 'age', 'parch']:
    print(f"Survived aggregation on class & {col}")
    display(pd.pivot_table(df, index='class', columns='survived', values=col, aggfunc=['mean', 'max']))

Survived aggregation on class & fare


mean                  max        
survived     alived       dead    alived    dead
class                                           
First     95.840984  64.684007  512.3292  263.00
Second    22.055700  19.412328   65.0000   73.50
Third     13.694887  13.669364   56.4958   69.55

Survived aggregation on class & age


mean               max     
survived     alived       dead alived dead
class                                     
First     34.373134  40.550000     80   71
Second    25.965517  33.134021     62   70
Third     22.731092  26.932796     63   74

Survived aggregation on class & parch


mean              max     
survived    alived      dead alived dead
class                                   
First     0.395522  0.300000      2    4
Second    0.643678  0.144330      3    2
Third     0.420168  0.384409      5    6

### Feature engineering
In this session, we will try to verify the affect of bin-group from the numerical variables (`age`, `fare`) to the response `survived`

In [3]:
df['age_group'] = pd.cut(df['age'], bins=5)
df['faregroup'] = pd.cut(df['fare'], bins=5)
for col in ['class', 'sex', 'embark_town', 'alone', 'age_group', 'faregroup']:
    display(pd.pivot_table(df, index='survived', columns=col, values='alive', aggfunc='count'))

class,First,Second,Third
survived,,,
alived,134,87,119
dead,80,97,372


sex,female,male
survived,,
alived,231,109
dead,81,468


embark_town,Cherbourg,Queenstown,Southampton
survived,,,
alived,93,30,217
dead,75,47,427


alone,False,True
survived,,
alived,179,161
dead,175,374


age_group,"(-0.08, 16.0]","(16.0, 32.0]","(32.0, 48.0]","(48.0, 64.0]","(64.0, 80.0]"
survived,,,,,
alived,55,181,74,29,1
dead,45,344,111,39,10


faregroup,"(-0.512, 102.466]","(102.466, 204.932]","(204.932, 307.398]","(307.398, 409.863]","(409.863, 512.329]"
survived,,,,,
alived,301,25,11,0,3
dead,535,8,6,0,0


There is not any difference between each bin-group to the final-output hence the `bingroup` are not neccessary here (`age_group` and `faregroup`)

In [4]:
y = df[['sex','embark_town','survived']]
X = df.drop(columns=["survived", "alive", "pclass", "age_group", "faregroup", "parch", 
                     "who", "sibsp", "adult_male", "deck", "embarked"])
X_train, X_valid, y_train, y_valid = train_test_split(X.drop(columns=['sex','embark_town']), y, test_size=0.3, 
                                                      stratify=y, 
                                                      random_state=42)
df_train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
df_valid = pd.concat([X_valid, y_valid], axis=1).reset_index(drop=True)
df_train.head()

,age,fare,class,alone,sex,embark_town,survived
0,20,7.8542,Third,True,male,Southampton,dead
1,28,25.4667,Third,False,female,Southampton,dead
2,21,7.8542,Third,True,male,Southampton,dead
3,28,7.3125,Third,True,male,Southampton,dead
4,36,13.0000,Second,True,female,Southampton,alived


Initialize `h2o`

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpex4e4p8c
  JVM stdout: /tmp/tmpex4e4p8c/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpex4e4p8c/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 12 days
H2O_cluster_name:,H2O_from_python_unknownUser_cpdqd7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Transform this to h2o-frame

In [6]:
h2o_train_df = h2o.H2OFrame(df_train)
h2o_valid_df = h2o.H2OFrame(df_valid)
h2o_train_df

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


age,fare,class,alone,sex,embark_town,survived
20,7.8542,Third,True,male,Southampton,dead
28,25.4667,Third,False,female,Southampton,dead
21,7.8542,Third,True,male,Southampton,dead
28,7.3125,Third,True,male,Southampton,dead
36,13,Second,True,female,Southampton,alived
31,26.25,Second,False,male,Southampton,dead
49,56.9292,First,False,male,Cherbourg,alived
23,7.2292,Third,True,male,Cherbourg,dead
25,30,Second,False,female,Southampton,alived
7,29.125,Third,False,male,Queenstown,dead


As a results from [Section 2.2.4 (clf-tradml-vs-automl.ipynb)](https://github.com/NhanDoV/Lectures_notes-teaching-in-VN-/blob/master/h2o_AutoML/classification/clf-tradml-vs-automl.ipynb), now we will focus on the GBM model

In [7]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

my_model = H2OGradientBoostingEstimator(ntrees=100, stopping_metric='auc',
                                        keep_cross_validation_predictions = True,
                                        stopping_rounds = 3,
                                        model_id = "first_model", seed = 1234,
                                        stopping_tolerance = 0.0005)
my_model.train(x = ["class","alone","age","fare","sex","embark_town"], y = "survived", 
               training_frame = h2o_train_df, 
               validation_frame = h2o_valid_df)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: first_model


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    6                  6                           1704                   5            5            5             16            19            18

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.15356659856582391
RMSE: 0.3918757437834395
LogLoss: 0.48849047101030363
Mean Per-Class Error: 0.1797312675070028
AUC: 0.8997833508403361
AUCPR: 0.9248738133267748
Gini: 0.7995667016806722

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5962821147344921
        alived    dead    Error    Rate
------  --------  ------  -------  -------------
alived  176       62      0.2605   (62.0/238.0)
dead    38        346     0.099    (38.0/384.0)
Total   214       408     0.1608   (100.0/622.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.596282     0.873737  28
max f2                       0.437787     0.934602  44
max f0point5                 0.641435     0.870879  22
max accuracy                 0.605223     0.840836  27
max precision                0.759321     1         0
max recall                   0.353874     1         47
max specificity              0.759321     1         0
max absolute_mcc             0.605223     0.661856  27
max min_per_class_accuracy   0.641435     0.823529  22
max mean_per_class_accuracy  0.605223     0.829558  27
max tns                      0.759321     238       0
max fns                      0.759321     373       0
max fps                      0.328924     238       50
max tps                      0.353874     384       47
max tnr                      0.759321     1         0
max fnr                      0.759321     0.971354  0
max fpr                      0.328924     1         50
max tpr                      0.353874     1         47

Gains/Lift Table: Avg response rate: 61.74 %, avg score: 61.65 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0176849                   0.759321           1.61979    1.61979            1                0.759321  1                           0.759321            0.0286458       0.0286458                  61.9792   61.9792            0.0286458
2        0.0498392                   0.75848            1.61979    1.61979            1                0.75848   1                           0.758778            0.0520833       0.0807292                  61.9792   61.9792            0.0807292
3        0.0675241                   0.757201           1.61979    1.61979            1                0.757201  1                           0.758365            0.0286458       0.109375                   61.9792   61.9792            0.109375
4        0.210611                    0.74854            1.52879    1.55797            0.94382          0.749134  0.961832                    0.752094            0.21875         0.328125                   52.8792   55.7968            0.307117
5        0.451768                    0.738067           1.40382    1.47568            0.866667         0.738284  0.911032                    0.744722     

In [8]:
my_model.model_id

'first_model'

### 2. What happend if we didnt use train_test_split from `sklearn`?

In [9]:
# split into train and validation sets
train, valid = h2o.H2OFrame(df.drop(columns=["alive", "pclass", "age_group", "faregroup", 
                                             "parch", "who", "sibsp", "adult_male", 
                                             "deck", "embarked"])).split_frame(ratios = [.7], 
                                                                               seed = 1234)
train

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


survived,sex,age,fare,class,embark_town,alone
alived,female,38,71.2833,First,Cherbourg,False
alived,female,26,7.925,Third,Southampton,True
dead,male,35,8.05,Third,Southampton,True
dead,male,28,8.4583,Third,Queenstown,True
dead,male,54,51.8625,First,Southampton,True
dead,male,2,21.075,Third,Southampton,False
alived,female,27,11.1333,Third,Southampton,False
alived,female,4,16.7,Third,Southampton,False
alived,female,58,26.55,First,Southampton,True
dead,male,20,8.05,Third,Southampton,True


In [10]:
my_model = H2OGradientBoostingEstimator(ntrees=100, stopping_metric='auc',
                                        keep_cross_validation_predictions = True,
                                        stopping_rounds = 3,
                                        model_id = "first_model", seed = 1234,
                                        stopping_tolerance = 0.0005)
my_model.train(x = ["class","alone","age","fare","sex","embark_town"], y = "survived", 
               training_frame = train, 
               validation_frame = valid)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: first_model


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    15                 15                          4382                   5            5            5             17            22            18.6

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1170673001067649
RMSE: 0.3421509902174256
LogLoss: 0.3938495857075096
Mean Per-Class Error: 0.1677128121436567
AUC: 0.9156179058259841
AUCPR: 0.9343343321265223
Gini: 0.8312358116519682

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46858711430793226
        alived    dead    Error    Rate
------  --------  ------  -------  ------------
alived  175       72      0.2915   (72.0/247.0)
dead    17        370     0.0439   (17.0/387.0)
Total   192       442     0.1404   (89.0/634.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.468587     0.892642  132
max f2                       0.297915     0.93046   159
max f0point5                 0.624654     0.893326  96
max accuracy                 0.624654     0.864353  96
max precision                0.888371     1         0
max recall                   0.221493     1         176
max specificity              0.888371     1         0
max absolute_mcc             0.624654     0.71671   96
max min_per_class_accuracy   0.67317      0.850202  88
max mean_per_class_accuracy  0.624654     0.860329  96
max tns                      0.888371     247       0
max fns                      0.888371     386       0
max fps                      0.127531     247       204
max tps                      0.221493     387       176
max tnr                      0.888371     1         0
max fnr                      0.888371     0.997416  0
max fpr                      0.127531     1         204
max tpr                      0.221493     1         176

Gains/Lift Table: Avg response rate: 61.04 %, avg score: 61.00 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.011041                    0.867744           1.63824   1.63824            1                0.877986  1                           0.877986            0.0180879       0.0180879                  63.8243   63.8243            0.0180879
2        0.0283912                   0.855503           1.63824   1.63824            1                0.857872  1                           0.865694            0.0284238       0.0465116                  63.8243   63.8243            0.0465116
3        0.0299685                   0.853691           1.63824   1.63824            1                0.855029  1                           0.865133            0.00258398      0.0490956                  63.8243   63.8243            0.0490956
4        0.0410095                   0.847898           1.63824   1.63824            1                0.852321  1                           0.861684            0.0180879       0.0671835                  63.8243   63.8243            0.0671835
5        0.0536278                   0.842462           1.63824   1.63824            1                0.844637  1                           0.857673    